In [1]:
import numpy as np

from Wolfe_line_search import wolfe_line_search

# ALgorithm

In [2]:
def smabfgs(f, grad_f, x0, tol=1e-6, max_iter=500):
    """
    Implementation of the SMABFGS algorithm as described in the paper.
    
    Parameters:
        f (callable): Objective function to minimize.
        grad_f (callable): Gradient of the objective function.
        x0 (numpy array): Initial guess.
        tol (float): Convergence tolerance.
        max_iter (int): Maximum number of iterations.
        
    Returns:
        x (numpy array): Optimized variable.
        f_val (float): Objective function value at the minimum.
        iter_count (int): Number of iterations performed.
    """
    x = x0
    g = grad_f(x)
    n = len(x0)
    I = np.eye(n)  # Identity matrix
    theta = 1.0    # Scaling parameter
    iter_count = 0

    while np.linalg.norm(g) > tol and iter_count < max_iter:
        iter_count += 1

        # Compute search direction
        H = theta * I
        p = -H @ g

        # Wolfe line search to determine step size
        alpha = wolfe_line_search(f, grad_f, x, p)

        # Update x
        x_new = x + alpha * p

        # Update gradient
        g_new = grad_f(x_new)

        # Compute differences
        s = x_new - x
        y = g_new - g

        # Update scaling parameter θ
        theta = np.dot(s, y) / np.dot(y, y)

        # Compute the rank-one modification term
        tau = np.dot(s, y) / np.linalg.norm(s)**2
        z = -theta * y + (1 + theta * np.dot(y, y) / np.dot(s, y)) * s
        gamma = tau + np.dot(s, y) / (np.linalg.norm(s)**2 + tau * theta * (np.linalg.norm(y)**2 / np.dot(s, y) - np.dot(s, y) / np.linalg.norm(s)**2))

        # Augmented Hessian update
        H += -tau / gamma * np.outer(z, z) / np.dot(s, y)

        # Prepare for next iteration
        x = x_new
        g = g_new

    return x, f(x), iter_count

# Simple Quadratic Function

In [3]:
# Example usage
if __name__ == "__main__":
    # Define the function and its gradient
    def f(x):
        return (x[0] - 1)**2 + (x[1] - 2)**2

    def grad_f(x):
        return np.array([2 * (x[0] - 1), 2 * (x[1] - 2)])

    # Initial guess
    x0 = np.array([2.0, 1.0])

    # Run the Broyden-CG algorithm with Wolfe line search
    x_opt, f_val, iterations = smabfgs(f, grad_f, x0)

    print(f"Optimized x: {x_opt}")
    print(f"Function value at minimum: {f_val}")
    print(f"Iterations: {iterations}")

Optimized x: [1. 2.]
Function value at minimum: 0.0
Iterations: 1


# Rosenbruck Function

In [5]:
# Define the Rosenbrock function and its gradient
def rosenbrock(x):
    return (1 - x[0])**2 + 100 * (x[1] - x[0]**2)**2

def grad_rosenbrock(x):
    grad_x0 = -2 * (1 - x[0]) - 400 * x[0] * (x[1] - x[0]**2)
    grad_x1 = 200 * (x[1] - x[0]**2)
    return np.array([grad_x0, grad_x1])

# Example usage
if __name__ == "__main__":
    # Initial guess
    x0 = np.array([-1.2, 1.0])

    # Run the Broyden-CG algorithm with Wolfe line search
    x_opt, f_val, iterations = smabfgs(rosenbrock, grad_rosenbrock, x0)

    print(f"Optimized x: {x_opt}")
    print(f"Function value at minimum: {f_val}")
    print(f"Iterations: {iterations}")

Optimized x: [0.99999995 0.99999989]
Function value at minimum: 2.8276664658719827e-15
Iterations: 213


# Exponential Function

In [6]:
import numpy as np

# Define the exponential test function
def exponential_function(x):
    """
    Exponential test function: f(x, y) = exp(x) + exp(y)
    """
    return np.exp(x[0]) + np.exp(x[1])

# Define the gradient of the exponential test function
def grad_exponential_function(x):
    """
    Gradient of the exponential test function: grad(f) = [exp(x), exp(y)]
    """
    grad_x0 = np.exp(x[0])
    grad_x1 = np.exp(x[1])
    return np.array([grad_x0, grad_x1])

# Example usage
if __name__ == "__main__":
    # Initial guess
    x0 = np.array([-1.0, -1.0])

    # Example function calls to hypothetical optimization methods
    # Ensure that `smabfgs`, `gradient_descent`, and `broyden_cg` are implemented or imported
    x_opt, f_val, iterations = smabfgs(exponential_function, grad_exponential_function, x0)

    print(f"Optimized x: {x_opt}")
    print(f"Function value at minimum: {f_val}")
    print(f"Iterations: {iterations}")

Optimized x: [-14.63565282 -14.63565282]
Function value at minimum: 8.807380048985129e-07
Iterations: 20


In [7]:
import numpy as np

# Define the non-convex test function
def non_convex_function(x):
    """
    Non-convex test function: f(x, y) = sin(x) * cos(y)
    """
    return np.sin(x[0]) * np.cos(x[1])

# Define the gradient of the non-convex test function
def grad_non_convex_function(x):
    """
    Gradient of the non-convex test function:
    grad(f) = [cos(x) * cos(y), -sin(x) * sin(y)]
    """
    grad_x0 = np.cos(x[0]) * np.cos(x[1])  # Partial derivative w.r.t. x
    grad_x1 = -np.sin(x[0]) * np.sin(x[1])  # Partial derivative w.r.t. y
    return np.array([grad_x0, grad_x1])

# Define the Hessian of the non-convex test function
def hess_non_convex_function(x):
    """
    Hessian of the non-convex test function:
    The second derivatives:
    H[0,0] = -sin(x) * cos(y)
    H[1,1] = -sin(x) * cos(y)
    H[0,1] = -cos(x) * sin(y)
    H[1,0] = -cos(x) * sin(y)
    """
    hess_x0x0 = -np.sin(x[0]) * np.cos(x[1])  # Second partial derivative w.r.t. x
    hess_x1x1 = -np.sin(x[0]) * np.cos(x[1])  # Second partial derivative w.r.t. y
    hess_x0x1 = -np.cos(x[0]) * np.sin(x[1])  # Mixed partial derivative
    hess_x1x0 = hess_x0x1  # Symmetry of the Hessian
    return np.array([[hess_x0x0, hess_x0x1], [hess_x1x0, hess_x1x1]])

# Example usage
if __name__ == "__main__":
    # Initial guess
    x0 = np.array([1.0, 1.0])

    # Example function calls to hypothetical optimization methods
    # Ensure that `smabfgs`, `gradient_descent`, and `broyden_cg` are implemented or imported
    x_opt, f_val, iterations = smabfgs(non_convex_function, grad_non_convex_function, x0)

    print(f"Optimized x: {x_opt}")
    print(f"Function value at minimum: {f_val}")
    print(f"Iterations: {iterations}")


Optimized x: [1.57079608 3.14159239]
Function value at minimum: -0.9999999999999368
Iterations: 7
